In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail()

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head()

,artist,song,text
0,Journey,If He Should Break Your Heart,Please be wrong \r\nPlease don't tell me it's...
1,Randy Travis,"Spirit Of A Boy, Wisdom Of A Man","He was 16, tender and tough. \r\nShe was too,..."
2,Dean Martin,Heart Over Mind,I love you so much that I can't leave you \r\...
3,Nina Simone,It Don't Mean A Thing,"Don't mean a thing, you can't swing \r\nDooa,..."
4,Bonnie Raitt,Everything That Touches You,You say you're all alone \r\nWith visions all...


In [9]:
df['text'][0]

"Please be wrong  \r\nPlease don't tell me it's really you  \r\nTell me eyes,tell my heart what I'm goin' through  \r\nI believed, I believed I was over you  \r\n  \r\nIf he should break your heart  \r\nI'll always know where you are  \r\nIf you should fall apart  \r\nI'll be there, I'll be around  \r\nIf he should break your heart  \r\nI'll be there  \r\n  \r\nIs this right  \r\nI don't care I'm alone with you  \r\nHere we are, once again you've got someone new  \r\nLonely hands, please don't do what you want to do  \r\n  \r\nIf he should break your heart  \r\nI'll always know where you are  \r\nIf you should fall apart  \r\nI'll be there, I'll be around  \r\n  \r\nIf he should break your heart  \r\nFrom my soul I say good-bye yesterday  \r\nOne more night alone, without the touch of you  \r\nThen I start to break is it my mistake  \r\nTell me am I wrong for holdin' on to you  \r\n  \r\nIf he should break your heart  \r\nI'll always know where you are  \r\nIf you should fall apart  \r

In [10]:
df.shape

(5000, 3)

In [11]:
# Text Preprocessing
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [12]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [19]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [20]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [23]:
similarity[0]

array([1.        , 0.03744962, 0.25468655, ..., 0.01339812, 0.01199637,
       0.01291713])

In [24]:
df[df['song'] == 'Crying Over You']

,artist,song,text
4133,ABBA,Crying Over You,i 'm waitin ' for you babi i 'm sit all alon i...


In [25]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [26]:
recommendation('Crying Over You')

['Blue Eyes Crying In The Rain',
 "Cryin' Time Again",
 "Cryin' In One Eye",
 'Put It On',
 'Lovesick Blues',
 "Blues For X'mas",
 'Lovesick Blues',
 'Blues Leave Me Alone',
 'Here I Am, Send Me',
 'Color Of The Blues',
 'You Move Me',
 'George Jackson',
 'He Was A Friend Of Mine',
 'The Battle Is The Lords',
 'Earth Blues',
 'Sonnet',
 'I Surrender',
 'Thank You Lord',
 'Blue Is Not A Word',
 'I Know']

In [27]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))